# Version 1
2022-10-13

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
project_id = 'tweetprediction'
bucket_name = 'ticks_with_indicators_with_volume'
bucket_name_tick_data = 'bittrex_tick_data'

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, ReLU, Add, Flatten, Concatenate, LayerNormalization, UpSampling2D, Activation, LSTM, Multiply, Dropout, Reshape, Permute, BatchNormalization, MaxPooling1D, AveragePooling1D, MaxPooling3D, AveragePooling2D, LayerNormalization, MaxPooling2D
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
try:
  import talib
except:
  #!cp /content/drive/MyDrive/Privat/Crypto/ta-lib-0.4.0-src.tar.gz ta-lib-0.4.0-src.tar.gz
  # !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
  #!tar -xzvf ta-lib-0.4.0-src.tar.gz
  !unzip /content/drive/MyDrive/Privat/Crypto/talibmaked.zip -d /
  %cd ta-lib
  #!./configure --prefix=/usr
  #!make
  !make install
  !pip install Ta-Lib
  
  import talib
  from talib import MA_Type

In [ ]:
!pip install transformers

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
import sys
sys.path.insert(0, "/content/drive/MyDrive/Privat/Crypto/CryptoCrystalBall/DataStreamCreator")
sys.path.insert(0, "/content/drive/MyDrive/Privat/Crypto/CryptoCrystalBall/IndicatorCalculator")

In [ ]:
from IndicatorCalculator import IndicatorCalculator, IndicatorCalculationError
import DataStreamCreator

In [ ]:
import pandas as pd
import numpy as np
import json
import copy
import pickle
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
from sklearn.cluster import Birch

In [ ]:
RANDOM_SEED = 10

In [ ]:
BASE_PATH = "/content/drive/MyDrive/Privat/Crypto/bittrex"
checkpoint_path = "gs://ticks_with_indicators_with_volume/chk/"

In [ ]:
# Get file names
from os import listdir
from os.path import isfile, join
VALID_CURRS_DIR = "/content/drive/MyDrive/Privat/Crypto/bittrex/train/normalCharts"
validcurrs = [f.replace(".jpg","") for f in listdir(VALID_CURRS_DIR) if (isfile(join(VALID_CURRS_DIR, f)))]
validcurrs = sorted(validcurrs)
len(validcurrs)

In [ ]:
# Get train file names
import os
from os import listdir
from os.path import isfile, join

TRAIN_PATH = os.path.join(BASE_PATH, "train")

trainfiles = [f for f in listdir(TRAIN_PATH) if isfile(join(TRAIN_PATH, f)) and ".csv" in f ]
trainfiles = [f for f in trainfiles if "-USDT" in f]
trainfiles = [f for f in trainfiles if f.split("/")[-1].replace(".csv","") in validcurrs]
trainfiles = sorted(trainfiles)
len(trainfiles)

# trainfiles = trainfiles[:20]
# len(trainfiles)

In [ ]:
TRAIN_PATH

In [ ]:
# Get test file names
import os
from os import listdir
from os.path import isfile, join

TEST_PATH = os.path.join(BASE_PATH, "test")

# testfiles = [f for f in listdir(TEST_PATH) if isfile(join(TEST_PATH, f)) and ".csv" in f ]
# testfiles = [f for f in testfiles if "-USDT" in f]
# testfiles = sorted(testfiles)

# indices_to_access = [100,7,43]
# accessed_mapping = map(testfiles.__getitem__, indices_to_access)
# testfiles = list(accessed_mapping)
testfiles = ['ETH-USDT.csv', 'BMP-USDT.csv', 'DOGE-USDT.csv', 'XRP-USDT.csv']
testfiles 

In [ ]:
# SMCNT = 200
# SMCNT2 = 200
# 0.4	0.4	0.0	2.000000e-01	2.000000e-01	

In [ ]:
BATCH_SIZE = 64
FEATURES = 228
PATTERN_CNT = 61
X_LOOKBACK_CNT = 128 #192 #64
Y_LOOKAHEAD_CNT = 0
GAIN_LOOKAROUND_CNT = np.nan

In [ ]:
# Y_TYPE = 3 # 0 for categorical, 1 for float, 2 for gain lookaround, 3 for entry/exit
# SMCNT = 48
# SMCNT2 = 48
# EXPECTED_GAIN_LOOKFORWARD = 2*24
# ENTR_THR = 0.6
# ENTR_THR2 = 0.6
# ENTR_THR3 = 0.0
# EXIT_THR = -0.1
# EXIT_THR2 = 0.1

In [ ]:
# PARAM_SET = 2

# Y_TYPE = 3 # 0 for categorical, 1 for float, 2 for gain lookaround, 3 for entry/exit
# SMCNT = 48
# SMCNT2 = 48
# EXPECTED_GAIN_LOOKFORWARD = 2*24
# ENTR_THR = 0.9
# ENTR_THR2 = 0.8
# ENTR_THR3 = 0.0
# EXIT_THR = -0.5
# EXIT_THR2 = 0.1

In [ ]:
# Parameterset 3
PARAM_SET = 3

Y_TYPE = 3 # 0 for categorical, 1 for float, 2 for gain lookaround, 3 for entry/exit
SMCNT = 48
SMCNT2 = 48
EXPECTED_GAIN_LOOKFORWARD = 7*24
ENTR_THR = 0.9
ENTR_THR2 = 0.8
ENTR_THR3 = 0.1
EXIT_THR = -0.75
EXIT_THR2 = -0.7

CLASS_WEIGHT = {0:3.17505149, 1: 1.62627684, 2: 1.0}


In [ ]:
# # # Parameterset 4

# Y_TYPE = 1 # 0 for categorical, 1 for float, 2 for gain lookaround, 3 for entry/exit
# SMCNT = 120
# SMCNT2 = 48
# EXPECTED_GAIN_LOOKFORWARD = 7*24
# ENTR_THR = 0.9
# ENTR_THR2 = 0.8
# ENTR_THR3 = 0.1
# EXIT_THR = -0.75
# EXIT_THR2 = -0.7

In [ ]:
SHORTSPAN = 6
MIDSPAN = 48
LONGSPAN = 120

In [ ]:
# FRQ_SMOOTH_PERIODS = [1,2,16]
# TIME_WARP_FACTORS = [1,4,16]

FRQ_SMOOTH_PERIODS = [1] #[1,4,8]
TIME_WARP_FACTORS = [2]

In [ ]:
# FRQ_SMOOTH_PERIODS = [1,4,8]
# TIME_WARP_FACTORS = [1]

In [ ]:
# # New periods
# FRQ_SMOOTH_PERIODS = [1,4,16,64]
# TIME_WARP_FACTORS = [1,4,8,16]

In [ ]:
import tensorflow as tf
from tensorflow.python.lib.io import file_io

# Visualization of y

In [ ]:
VISUAL_ID = 7 #7

In [ ]:
y_dir = None
y_dir2nd = None

ite = DataStreamCreator.FileListToDataStream([trainfiles[VISUAL_ID]], 1, TRAIN_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                              exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                              parallel_generators = 1, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              shuffle = False,
                                              batch_norm = False)
while True:
  try:    
    ne = next(ite)
  except StopIteration:
    break
  if y_dir is None or y_dir2nd is None:
    y_dir = ne[1][:,0]
    y_dir2nd = ne[1][:,1]
  else:
    y_dir = np.concatenate([y_dir, ne[1][:,0]])
    y_dir2nd = np.concatenate([y_dir2nd, ne[1][:,1]])

In [ ]:
# Load the tick table
tickpath = os.path.join(TRAIN_PATH, trainfiles[VISUAL_ID])
tickdata = pd.read_csv(tickpath)
tickdata.set_index("startsAt", inplace = True)
tickdata.sort_index()
 
# Crop the lookback
tickdata = tickdata.iloc[X_LOOKBACK_CNT:]

tickdata

In [ ]:
y_dir.shape

In [ ]:
y_dir

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))

ax1.plot(y_dir2nd, color="red")
ax1.plot(y_dir, color="green")

# ax1.set_xlim(0,1000)

ax2 = ax1.twinx()

y_dir_naned = copy.deepcopy(y_dir)
y_dir_naned[y_dir_naned == 0] = np.nan

exit_naned = copy.deepcopy(y_dir2nd)
exit_naned[exit_naned == 0] = np.nan

# ax2.plot(tickdata.loc[:,"close"].values * y_dir_naned, color = "green", linewidth=2, marker="X", markersize=15)
# ax2.plot(tickdata.loc[:,"close"].values * exit_naned, color = "red", linewidth=2, marker="X", markersize=15)

ax2.plot(tickdata.loc[:,"close"].values, color = "black")

ax1.set_ylim(0,1.1)

# Birch training

In [ ]:
stop

In [ ]:
ite = DataStreamCreator.FileListToDataStream(trainfiles, 32, TRAIN_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                              exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                              parallel_generators = 32, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              norm_price_related_indicators = False,
                                              shuffle = True,
                                              batch_norm = False)

X_raw = np.zeros((10032,128,228))
i = 0

while True:
  ne = next(ite)

  X_raw[i:i+ne[0].shape[0], :, :] = ne[0]
  i = i+ne[0].shape[0]

  print(i)
  if 10000 < i:
    break

In [ ]:
X_raw.shape

In [ ]:
# X_raw = X_raw[:210,:,:]
# X_raw.shape

In [ ]:
plt.plot(X_raw[204,90,:])
plt.plot(X_raw[205,90,:])
plt.plot(X_raw[208,90,:])

In [ ]:
X_raw.shape[0] * X_raw.shape[1] * X_raw.shape[2]

In [ ]:
np.sum(X_raw >= 100) / (X_raw.shape[0] * X_raw.shape[1] * X_raw.shape[2])

In [ ]:
np.sum(X_raw < 100)

In [ ]:
def NormForBirch(XIn):
  _THR = 100
  normed = copy.deepcopy(XIn)

  print(f"Elements over threshold: {np.sum(normed > _THR) / (normed.shape[0] * normed.shape[1] * normed.shape[2])}")
  print(f"Elements under threshold: {np.sum(normed < -_THR) / (normed.shape[0] * normed.shape[1] * normed.shape[2])}")

  normed[normed >= _THR] = _THR
  normed[normed <= -_THR] = -_THR
  # normed = np.tanh(normed)
  # normed += _THR + 1
  # normed = np.log10(normed)
  # normed -= np.log10(_THR + 1)
  
  return normed

In [ ]:
X_normed = NormForBirch(X_raw)
X_normed.shape

In [ ]:
del X_raw

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))
_ = ax1.plot(X_normed[204,90,:])
_ = ax1.plot(X_normed[205,90,:])
_ = ax1.plot(X_normed[208,90,:])

In [ ]:
import random

In [ ]:
# randomElementsLen = int(X_raw_RS.shape[0] * 0.5)
# print(("randomElementsLen", randomElementsLen))
# randomIndices = np.empty((randomElementsLen), dtype=int)

# for i in range(len(randomIndices)):
#   rn = int(random.randrange(0,X_raw_RS.shape[0]-1,1))

#   while rn in randomIndices:
#     rn = int(random.randrange(0,X_raw_RS.shape[0]-1,1))
  
#   randomIndices[i] = rn

In [ ]:
# randomElements = X_raw_RS[randomIndices,:]
# randomElements.shape

In [ ]:
np.min(X_normed)

In [ ]:
np.max(X_normed)

In [ ]:
X_normed_flat = np.reshape(X_normed, (X_normed.shape[0] * X_normed.shape[1], X_normed.shape[2]))
X_normed_flat.shape

In [ ]:
del X_normed

In [ ]:
brc = Birch(n_clusters=256-4) # Minus 4 because BOS,EOS,PAD and MASK Tokens are required for NLP model
brc.fit(X_normed_flat)
brc

In [ ]:
p = brc.predict(X_normed_flat)
p.shape

In [ ]:
p[:25]

In [ ]:
plt.plot(p)

In [ ]:
plt.hist(p)

In [ ]:
np.max(p)

In [ ]:
categoryDict = {}

# Create empty lists
for i in range(np.max(p)+1):
  categoryDict[i] = []

for i, xClass in enumerate(p):
  categoryDict[xClass].append(X_normed_flat[i,:])

In [ ]:
len(categoryDict)

In [ ]:
# Create and average feature map for each xClass
meanFeatureArray = np.zeros( (len(categoryDict), X_normed_flat.shape[-1]))

for xClass in categoryDict:
  print((xClass, len(categoryDict[xClass])))

  cdList = categoryDict[xClass]
  cdArray = np.array(cdList)

  meanFeature = np.mean(cdArray, axis=0)

  meanFeatureArray[xClass, :] = meanFeature

In [ ]:
meanFeatureArray.shape

In [ ]:
# plt.plot(cdArray)

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,10))
ax1.set_ylim(-1,1)
_ = ax1.plot(meanFeatureArray.transpose())

In [ ]:
meanFeatureArray.shape

In [ ]:
# Show a distance matrix to check if the clusters are different 
distances = np.zeros((meanFeatureArray.shape[0], meanFeatureArray.shape[0]))
for i in range(meanFeatureArray.shape[0]):
  for k in range(meanFeatureArray.shape[0]):
    distances[i,k,] = np.linalg.norm( meanFeatureArray[i,:]-meanFeatureArray[k,:] )

print(np.min(distances))
print(np.max(distances))

fig, ax1 = plt.subplots(figsize=(15,15))
_ = ax1.imshow(distances, cmap="gray")

### Save the clusterer

In [ ]:
sys.setrecursionlimit(10000) 
with file_io.FileIO("/content/brc.pkl", mode='wb+') as handle:
  pickle.dump(brc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("/content/brc.pkl", 'rb') as pickle_file:
  brc2 = pickle.load(pickle_file)

In [ ]:
X_normed_flat.shape

In [ ]:
p2 = brc.predict(X_normed_flat[:1000,:])
p2.shape

In [ ]:
# Check if the re-loaded prediction is the same as the original one
diff = p2 - p[:1000]
np.sum(np.abs(diff))

In [ ]:
# Store to drive
stop
!cp /content/brc.pkl /content/drive/MyDrive/Privat/Crypto/birchNew.pkl

# Train a neural network to predict the categories - NOT WORKING

In [ ]:
stop

In [ ]:
# Load the BIRCH categorizer
with open("/content/drive/MyDrive/Privat/Crypto/birchNew.pkl", 'rb') as pickle_file:
  brc = pickle.load(pickle_file)

In [ ]:
ite = DataStreamCreator.FileListToDataStream(trainfiles, 32, TRAIN_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              expected_gain_lookforward=EXPECTED_GAIN_LOOKFORWARD, entr_thr=ENTR_THR, entr_thr2=ENTR_THR2, entr_thr3=ENTR_THR3,
                                              exit_thr=EXIT_THR, exit_thr2=EXIT_THR2,
                                              parallel_generators = 1, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              norm_price_related_indicators = False,
                                              shuffle = True)

while True:
  ne = next(ite)

  print(ne[0].shape)

  break

In [ ]:
def trainGeneratorMethod():
  it = DataStreamCreator.FileListToDataStream(trainfiles, BATCH_SIZE, TRAIN_PATH,
                                              smooth_cnt=SMCNT, smooth_cnt2=SMCNT2,
                                              X_lookback_cnt = X_LOOKBACK_CNT,
                                              y_lookahead_cnt = Y_LOOKAHEAD_CNT,
                                              gain_lookaround_cnt = GAIN_LOOKAROUND_CNT,
                                              shortspan=SHORTSPAN, midspan=MIDSPAN, longspan=LONGSPAN,
                                              y_type = Y_TYPE,
                                              parallel_generators = 8, random_seed=RANDOM_SEED,
                                              y_exponent = 1.0,
                                              norm_price_related_indicators = False,
                                              shuffle=True)
  while True:
    try:
      ne = next(it)
      _X = ne[0]

      # Merge batch and time dimension, as birch take 2D data
      bIn = _X.reshape((_X.shape[0] * _X.shape[1], _X.shape[2]))
      bIn = NormForBirch(bIn)
      
      bOut = brc.predict(bIn)

      # Each feature-frame shall be converted into a number
      _y = bOut.reshape(_X.shape[0], _X.shape[1])

      # Limit the range to [0,1]
      # _y = _y / 1024.0

      # Create sparse matrices (one for the category, the other a binary repesantion)
      categoryIndices= np.round(_y).astype(int)
      ySparse = np.zeros((categoryIndices.shape[0], categoryIndices.shape[1], 1024), dtype=int)
      yBinSparse = np.zeros((categoryIndices.shape[0], categoryIndices.shape[1], 10), dtype=int)

      for i in range(categoryIndices.shape[0]):
        for j in range(categoryIndices.shape[1]):

          ySparse[i,j,categoryIndices[i,j]] = 1

          # Convert the index into a binary representation
          # categoryIndexBinString = bin(categoryIndices[i,j])[2:]
          # cropped = np.array([*categoryIndexBinString]).astype(int)
          # categoryIndexBin = np.zeros((10))
          # categoryIndexBin[-cropped.shape[0]:] = cropped
          # yBinSparse[i,j,:] = categoryIndexBin
      
      # yield (_X, ySparse, yBinSparse)
      yield (_X, ySparse)
    except StopIteration as si:
      logging.warning("StopIteration in FileListToDataStream")
      logging.warning(si)
      return

tfgenTrain = tf.data.Dataset.from_generator(trainGeneratorMethod,
                                            output_types = (tf.float32, tf.int64),
                                            output_shapes=(
                                                (BATCH_SIZE,X_LOOKBACK_CNT,228), 
                                                (BATCH_SIZE,X_LOOKBACK_CNT,1024)#,
                                                # (BATCH_SIZE,X_LOOKBACK_CNT,10)
                                                )
                                            )

tfgenTrain = tfgenTrain.prefetch(tf.data.AUTOTUNE)
tfgenTrain

n = next(tfgenTrain.as_numpy_iterator())
print(n[0].shape) # Normal X data
print(n[1].shape) # The one-hot encoded desired filter for each feature frame
# print(n[2].shape) # category id in binary

In [ ]:
from sklearn.utils import shuffle
import numpy as np

In [ ]:
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from datetime import datetime

# Create Model

In [ ]:
#@title CreateModelFeatureEmbeddingNew
# def CreateModelFeatureEmbeddingNew():

#   # Build your model input
#   inputTimeline = Input(shape=(X_LOOKBACK_CNT, FEATURES), name='input', dtype='float32')
  
#   # Convert feeatures into "word embeddings"
#   categoryFilters = Conv1D(filters=1024,
#                            kernel_size=1,
#                            padding="same",
#                            activation="softmax",
#                            data_format="channels_last",
#                            trainable=False,
#                            name="CategoryFilters")(inputTimeline)

#   # Convert the features into disrete integers, e.g. a "sentence"
#   floatFeatures = Dense(1, name="OneHotToFloat", trainable=False)(categoryFilters)

#   outputs = [floatFeatures]

#   # And combine it all in a model object
#   model = Model(inputs=inputTimeline, outputs=outputs, name='ModelFeatureEmbeddingNew')

#   # Set the weights for one hot to float conversion manually
#   manualWeights = np.array(range(1024))
#   manualWeights = np.expand_dims(manualWeights, axis=1)
#   manualBias = np.array([0])
#   model.get_layer("OneHotToFloat").set_weights([manualWeights, manualBias])

#   return model

# modelFeatureEmbeddingNew = CreateModelFeatureEmbeddingNew()

# Load the prepared model from drive
with tf.device('/CPU:0'):
  modelFeatureEmbeddingNew = tf.keras.models.load_model('/content/drive/MyDrive/Privat/Crypto/ModelFeatureEmbeddingNew.h5')
  modelFeatureEmbeddingNew.summary()

In [ ]:
##@title CreateModelMergeFeatureToOneFloat
# Inspired by https://towardsdatascience.com/how-to-use-convolutional-neural-networks-for-time-series-classification-56b1b0a07a57
def CreateModelMergeFeatureToOneFloat():

  # Build your model input
  inputTimeline = Input(shape=(X_LOOKBACK_CNT, FEATURES), name='input', dtype='float32')
  
  # Convert feeatures into "word embeddings"
  categoryFilters = Conv1D(filters=1024,
                           kernel_size=1,
                           padding="same",
                           activation="softmax",
                           data_format="channels_last",
                           name="CategoryFilters")(inputTimeline)

  binaryCategoryId = Dense(10, name="BinaryCategoryIdDense1")(categoryFilters)
  binaryCategoryId = Dense(10, name="BinaryCategoryIdDense2", activation="sigmoid")(binaryCategoryId)

  outputs = [categoryFilters]

  mnamesuffix = "_1"

  # And combine it all in a model object
  model = Model(inputs=inputTimeline, outputs=outputs, name='MergeFeatureToOneFloat'+mnamesuffix)

  return model

# modelFeatureToNormedInt = CreateModelMergeFeatureToOneFloat()
# modelFeatureToNormedInt.summary(line_length=170)

# Train the model

In [ ]:
model = CreateModelMergeFeatureToOneFloat()

In [ ]:
CHKPNT_NAME = "{}_T{}_{}LB_LF{}_EN{}_{}_{}_EX{}_{}_SM{}_{}".format(model.name, Y_TYPE, X_LOOKBACK_CNT, EXPECTED_GAIN_LOOKFORWARD, ENTR_THR, ENTR_THR2, ENTR_THR3, EXIT_THR, EXIT_THR2, SMCNT, SMCNT2)
CHKPNT_NAME

# Train

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=1e-05,
    epsilon=1e-05,
    decay=0.01)

In [ ]:
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = ['categorical_crossentropy'], 
    metrics=['mse', 'mae'])

# model.compile(
#     optimizer = optimizer,
#     loss = ['mse'], 
#     metrics=['mae'])

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# # !gcloud config set project {project_id} -y
# !gsutil cp -R gs://ticks_with_indicators_with_volume/logs/ModelNLP_1_T3_128LB_LF168_EN0.9_0.8_0.1_EX-0.75_-0.7_SM48_4820220915-162219 /content/logs

In [ ]:
# %tensorboard --logdir /content/logs

In [ ]:
# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances

In [ ]:
2**14 -1

In [ ]:
model.summary(line_length=220)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
  def __init__(self, save_freq, val_freq, checkpoint_path, model_name, epoch_add=0):
    self.save_freq = save_freq
    self.val_freq = val_freq
    self.checkpoint_path = checkpoint_path
    self.model_name = model_name
    self.current_epoch = 0
    self.epoch_add = epoch_add

  def on_epoch_begin(self, epoch, logs=None):
    self.current_epoch = epoch + self.epoch_add
    # keys = list(logs.keys())
    # print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

  def on_epoch_end(self, logs=None):
    self.saveTheModel("end", logs)

  def on_train_batch_end(self, batch, logs=None):
    self.saveTheModel(batch, logs)

  def saveTheModel(self, batch, logs=None):
    if (0 < batch and 0 == batch % self.save_freq) or ("end" == batch):
      if "end" == batch:
        _save_folder = os.path.join(self.checkpoint_path,
                                    self.model_name,
                                    "cp_daily_valid_{:01d}_end".format(self.current_epoch)
                                    )
      else:
        _save_folder = os.path.join(self.checkpoint_path,
                                    self.model_name,
                                    "cp_daily_valid_{:01d}_{:05d}".format(self.current_epoch, batch)
                                    )

      _model_path_local = os.path.join("/content/", "model.h5")
      _model_path_bucket = os.path.join(_save_folder, "model.h5")

      model.save(_model_path_local)
     
      # Copy model.h5 over to Google Cloud Storage
      with file_io.FileIO(_model_path_local, mode='rb') as input_f:
          with file_io.FileIO(_model_path_bucket, mode='wb+') as output_f:
              output_f.write(input_f.read())
              print("\nSaved model to: '" + _model_path_bucket + "'")

      # Save optimizer config
      c = copy.deepcopy(self.model.optimizer.get_config())

      fp = os.path.join(_save_folder, "c.pickle")
      with file_io.FileIO(fp, mode='wb+') as handle:
        pickle.dump(c, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Saved optimizer config to: '" + fp + "'")

      # Save optimizer weights
      w = copy.deepcopy(self.model.optimizer.get_weights())

      fp = os.path.join(_save_folder, "w.pickle")
      with file_io.FileIO(fp, mode='wb+') as handle:
        pickle.dump(w, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Saved optimizer weights to: '" + fp + "'")

    # if 0 < batch and 0 == batch % self.val_freq:
    #   print("-------------------------EVAL-------------------------")
    #   model.evaluate(tfgenTest)
    #   print("\n-------------------------EVAL-------------------------")

In [ ]:
epoch_add = 0

In [ ]:
CALLBACK_EVERY_N_BATCHES = 2000

In [ ]:
cc = CustomCallback(checkpoint_path = checkpoint_path,
                    model_name = CHKPNT_NAME,
                    save_freq = CALLBACK_EVERY_N_BATCHES,
                    val_freq = CALLBACK_EVERY_N_BATCHES,
                    epoch_add = epoch_add)

In [ ]:
import datetime
log_dir = "gs://ticks_with_indicators_with_volume/logs/" + CHKPNT_NAME + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
     histogram_freq=0,
     update_freq=CALLBACK_EVERY_N_BATCHES 
     )

In [ ]:
model.fit(tfgenTrain, epochs=200, verbose = 1)

In [ ]:
# 4602/Unknown - 8547s 2s/step - loss: 74.7547 - mse: 0.0018 - mae: 0.0019
# model.save("/content/drive/MyDrive/Privat/Crypto/FeatureEmbedding.h5")

In [ ]:
n = next(tfgenTrain.as_numpy_iterator())

In [ ]:
n[0].shape

In [ ]:
n[1].shape

In [ ]:
plt.plot(model.layers[1].get_weights()[0][0,:,1000])

In [ ]:
p = model.predict(n[0])
p.shape

In [ ]:
np.array(range(1024))

In [ ]:
tf.convert_to_tensor(np.array(range(1024)))

In [ ]:
cats = []
for i in range(p.shape[1]):
  cats.append(int(np.sum(p[0,i,:] * np.array(range(1024)))))
  print(cats[-1])

In [ ]:
plt.hist(cats)

In [ ]:
np.sum(p[0,4,:] * np.array(range(1024)))

In [ ]:
np.sum(p[0,5,:] * np.array(range(1024)))

In [ ]:
plt.plot(p[0,5,:])

In [ ]:
plt.plot(p[0,:])
plt.plot(n[1][0,:])

In [ ]:
p.shape

In [ ]:
X_LOOKBACK_CNT

In [ ]:
FEATURES

# Create a dense layer to convert one hot to int

In [ ]:
##@title CreateModelOneHotToInt
def CreateModelOneHotToInt():

  # Build your model input
  inputTimeline = Input(shape=(X_LOOKBACK_CNT, 1024), name='input', dtype='float32')

  floatFeature = Dense(1, name="OneHotToFloat")(inputTimeline)

  outputs = [floatFeature]
  
  # And combine it all in a model object
  model = Model(inputs=inputTimeline, outputs=outputs, name='ModelOneHotToInt')

  return model

modelOneHotToInt = CreateModelOneHotToInt()
modelOneHotToInt.summary(line_length=170)

In [ ]:
X_data = np.empty((1024,128,1024))
y_data = np.empty((1024,128,1))

for i in range(1024):
  X_data[i,:,i] = 1
  y_data[i,:,0] = i

In [ ]:
y_data

In [ ]:
X_data

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=1e-05,
    epsilon=1e-05,
    decay=0.01)

In [ ]:
# Compile the model
modelOneHotToInt.compile(
    optimizer = optimizer,
    loss = ['mse'], 
    metrics=['mae'])

In [ ]:
modelOneHotToInt.fit(X_data, y_data, epochs=100)

In [ ]:
X_test = np.zeros((1,128,1024))
X_test[:,0,2] = 1
X_test

In [ ]:
manualWeights = np.array(range(1024))
manualWeights = np.expand_dims(manualWeights, axis=1)
manualBias = np.array([0])
manualWeights.shape

In [ ]:
modelOneHotToInt.get_layer("OneHotToFloat").set_weights([manualWeights, manualBias])

In [ ]:
modelOneHotToInt.get_layer("OneHotToFloat").get_weights()[1].shape